In [23]:
import torch
import torchvision
import gym

import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import gym
from gym.envs.registration import register
#register(
#    id='FrozenLakeNotSlippery-v0',
#    entry_point='gym.envs.toy_text:FrozenLakeEnv',
#    kwargs={'map_name' : '4x4', 'is_slippery': False},
#    max_episode_steps=100,
#    reward_threshold=0.78, # optimum = .8196
#)

#env = gym.make('FrozenLake8x8-v0')
env = gym.make('FrozenLake-v0')
#env = gym.make('FrozenLakeNotSlippery-v0')
env.render()
class pi_net(nn.Module):
    def __init__(self):
        super(pi_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4 + 1)
        

    def forward(self, x):
        #print(x.shape)
        #print(x)
        x = F.relu(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = self.linear2(x)
        x = x.view(-1,5)
        #print(x.shape)
        #print(x)
        actions = x[:,:4]
        value = x[:,4]
        return actions, value
    
    


SFFF
FHFH
FFFH
HFFG


In [24]:
# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        #m.weight.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)
        m.weight.data.fill_(0.5)

import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical
mse = nn.MSELoss()

NUM_EPISODES = 1000000
GAMMA = 0.9
net = pi_net()
net.apply(weights_init)
optimizer = optim.RMSprop(net.parameters(), lr=0.0001)


score = []
times_trained = 0
times_reach_goal = 0
for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    episode_series = []
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = torch.FloatTensor(np_observation) 
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        action_probs, value = net(observation_tensor)
        action_probs_orig = action_probs
        #print("action_probs after net")
        #print(action_probs)
        #FOR EXPLORATION: 
        #action_probs = F.dropout(action_probs, p=0.3, training=True)
        #print("action_probs after dropout")
        #print(action_probs)
        action_probs = F.softmax(action_probs, dim=1)
        #print("action_probs after softmax")
        #print(action_probs)
        #action = action_probs.multinomial(num_samples=1)
        m = Categorical(action_probs)
        action = m.sample()
        
        
        
        #print("after eval")
        #print("action_probs")
        #print(action_probs)
        log_prob = m.log_prob(action)
        #print("log_prob")
        #print(log_prob)
        #break
        #print("softmax")
        #print(action_probs)
        #print("action")
        #print(str(action.item()))
        #print(type(prob.multinomial))
        
        #break
        # Execute action in environment.
        
        if k%10000 == 0:
            print("action_probs_orig ")
            print(action_probs_orig)
            print("On state="+ str(observation) + ", selected action=" + str(action.item()) + " , ")
        
        observation, reward, done, info = env.step(action.item()) 
        
        if k%10000 == 0:
            print("new state="+ str(observation) + ", done="+str(done))
        #if done and reward != 1.0:
        #    reward = -1.0
        step_data = [observation, action,log_prob, value, reward, done, info]
        episode_series.append(step_data)
        #env.render()
        
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: {}. Times reached goal: {}.".format(k, len(episode_series), reward, np.mean(score), times_trained, times_reach_goal))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        

    
    discounted_rewards = []
    actions = []
    values = []
    log_probs = []
    
    R = 0
    for i in reversed(range(len(episode_series))): # Build the different containers: action, value, reward
        [observation, action, log_prob, value, reward, done, info] = episode_series[i]
        R = reward + R * GAMMA
        discounted_rewards.append(R)
        actions.append(action)
        values.append(value)
        log_probs.append(log_prob)
        
    value_loss = 0
    policy_log_loss = 0
    for action, value, log_prob, reward in zip(actions, values, log_probs, discounted_rewards):
        reward_diff = reward - value.item() # Treat critic value as baseline
        #action.reinforce(reward_diff) # Try to perform better than baseline
        policy_log_loss += -log_prob * reward_diff
        value_loss += mse(value, torch.Tensor([reward])) # Compare with actual reward
        
    
    
    
    if R > 0.0 or True: # Optimize only if rewards are non zero.
        #print "Reward list"
        #print rewards_list
        optimizer.zero_grad()
        nodes = [value_loss] + [policy_log_loss]
        gradients = [torch.ones(1)] + [torch.ones(1)] # No gradients for reinforced values
        autograd.backward(nodes, gradients)
        optimizer.step()
        times_trained = times_trained + 1
    
    if R > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    

action_probs_orig 
tensor([[ 3.3964,  3.3324,  3.4507,  3.6962]])
On state=0, selected action=2 , 
new state=1, done=False
action_probs_orig 
tensor([[ 6.7988,  6.7349,  6.8532,  7.0986]])
On state=1, selected action=1 , 
new state=2, done=False
action_probs_orig 
tensor([[ 11.1356,  11.0716,  11.1900,  11.4354]])
On state=2, selected action=3 , 
new state=1, done=False
action_probs_orig 
tensor([[ 6.7988,  6.7349,  6.8532,  7.0986]])
On state=1, selected action=1 , 
new state=2, done=False
action_probs_orig 
tensor([[ 11.1356,  11.0716,  11.1900,  11.4354]])
On state=2, selected action=3 , 
new state=3, done=False
action_probs_orig 
tensor([[ 16.1356,  16.0716,  16.1900,  16.4354]])
On state=3, selected action=0 , 
new state=7, done=True
Episode 0 finished after 6 timesteps with r=0.0. Running score: 0.0. Times trained: 0. Times reached goal: 0.
Episode 1000 finished after 4 timesteps with r=0.0. Running score: 0.01. Times trained: 1000. Times reached goal: 13.
Episode 2000 finished a

Episode 42000 finished after 10 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 33.
Episode 43000 finished after 6 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 31.
Episode 44000 finished after 2 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 25.
Episode 45000 finished after 4 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 16.
Episode 46000 finished after 7 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 35.
Episode 47000 finished after 3 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 36.
Episode 48000 finished after 2 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 20.
Episode 49000 finished after 3 timesteps with r=0.0. Running score: 0.01. Times trained: 1000. Times reached goal: 29.
action_probs_orig 
tensor([[ 1.1751,  2.2906,  

Episode 83000 finished after 2 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 42.
Episode 84000 finished after 2 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 45.
Episode 85000 finished after 7 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 41.
Episode 86000 finished after 5 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 41.
Episode 87000 finished after 2 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 33.
Episode 88000 finished after 8 timesteps with r=0.0. Running score: 0.06. Times trained: 1000. Times reached goal: 48.
Episode 89000 finished after 12 timesteps with r=0.0. Running score: 0.01. Times trained: 1000. Times reached goal: 36.
action_probs_orig 
tensor([[ 0.8080,  2.7033,  2.5863,  0.3624]])
On state=0, selected action=1 , 
new state=4, done=False
action_probs_orig 
tensor([[ 0.9213,  3.082

Episode 131000 finished after 8 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 35.
Episode 132000 finished after 5 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 49.
Episode 133000 finished after 8 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 51.
Episode 134000 finished after 8 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 38.
Episode 135000 finished after 7 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 34.
Episode 136000 finished after 3 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 33.
Episode 137000 finished after 6 timesteps with r=0.0. Running score: 0.06. Times trained: 1000. Times reached goal: 48.
Episode 138000 finished after 3 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 37.
Episode 139000 finished after 12 timeste

action_probs_orig 
tensor([[-1.1258,  3.3118,  2.2872, -2.1427]])
On state=0, selected action=1 , 
new state=0, done=False
action_probs_orig 
tensor([[-1.1258,  3.3118,  2.2872, -2.1427]])
On state=0, selected action=1 , 
new state=1, done=False
action_probs_orig 
tensor([[-1.1355,  3.3774,  2.3402, -2.1657]])
On state=1, selected action=1 , 
new state=2, done=False
action_probs_orig 
tensor([[-1.1614,  3.5098,  2.4518, -2.1926]])
On state=2, selected action=1 , 
new state=3, done=False
action_probs_orig 
tensor([[-1.1959,  3.6776,  2.5946, -2.2215]])
On state=3, selected action=2 , 
new state=7, done=True
Episode 180000 finished after 5 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 43.
Episode 181000 finished after 5 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 49.
Episode 182000 finished after 7 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 47.
Episode 183000 finished af

...

Episode 801000 finished after 7 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 38.
Episode 802000 finished after 11 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 56.
Episode 803000 finished after 5 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 52.
Episode 804000 finished after 4 timesteps with r=0.0. Running score: 0.06. Times trained: 1000. Times reached goal: 54.
Episode 805000 finished after 5 timesteps with r=0.0. Running score: 0.07. Times trained: 1000. Times reached goal: 50.
Episode 806000 finished after 4 timesteps with r=0.0. Running score: 0.06. Times trained: 1000. Times reached goal: 47.
Episode 807000 finished after 6 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 46.
Episode 808000 finished after 10 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 43.
Episode 809000 finished after 4 timest

Episode 848000 finished after 4 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 55.
Episode 849000 finished after 2 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 49.
action_probs_orig 
tensor([[-4.1386,  7.7649, -2.0288, -4.4265]])
On state=0, selected action=1 , 
new state=4, done=False
action_probs_orig 
tensor([[-4.3024,  8.1199, -2.0097, -4.5855]])
On state=4, selected action=1 , 
new state=5, done=True
Episode 850000 finished after 2 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 47.
Episode 851000 finished after 2 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 49.
Episode 852000 finished after 3 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 49.
Episode 853000 finished after 11 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 60.
Episode 854000 finished after 3 ti

Episode 891000 finished after 2 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 44.
Episode 892000 finished after 2 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 56.
Episode 893000 finished after 5 timesteps with r=0.0. Running score: 0.07. Times trained: 1000. Times reached goal: 49.
Episode 894000 finished after 7 timesteps with r=0.0. Running score: 0.01. Times trained: 1000. Times reached goal: 48.
Episode 895000 finished after 4 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 56.
Episode 896000 finished after 4 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 52.
Episode 897000 finished after 10 timesteps with r=0.0. Running score: 0.08. Times trained: 1000. Times reached goal: 51.
Episode 898000 finished after 5 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 44.
Episode 899000 finished after 2 timeste

Episode 937000 finished after 12 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 47.
Episode 938000 finished after 4 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 58.
Episode 939000 finished after 3 timesteps with r=0.0. Running score: 0.03. Times trained: 1000. Times reached goal: 51.
action_probs_orig 
tensor([[-4.1561,  7.9657, -2.2269, -4.3918]])
On state=0, selected action=1 , 
new state=0, done=False
action_probs_orig 
tensor([[-4.1561,  7.9657, -2.2269, -4.3918]])
On state=0, selected action=1 , 
new state=0, done=False
action_probs_orig 
tensor([[-4.1561,  7.9657, -2.2269, -4.3918]])
On state=0, selected action=1 , 
new state=1, done=False
action_probs_orig 
tensor([[-4.1661,  7.9898, -2.2312, -4.4036]])
On state=1, selected action=1 , 
new state=2, done=False
action_probs_orig 
tensor([[-4.1760,  8.0138, -2.2355, -4.4154]])
On state=2, selected action=1 , 
new state=3, done=False
action_probs_orig 
tenso

Episode 985000 finished after 2 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 35.
Episode 986000 finished after 2 timesteps with r=0.0. Running score: 0.05. Times trained: 1000. Times reached goal: 47.
Episode 987000 finished after 5 timesteps with r=0.0. Running score: 0.06. Times trained: 1000. Times reached goal: 45.
Episode 988000 finished after 6 timesteps with r=0.0. Running score: 0.02. Times trained: 1000. Times reached goal: 33.
Episode 989000 finished after 7 timesteps with r=0.0. Running score: 0.04. Times trained: 1000. Times reached goal: 58.
action_probs_orig 
tensor([[-4.1373,  8.1354, -2.3332, -4.4445]])
On state=0, selected action=1 , 
new state=1, done=False
action_probs_orig 
tensor([[-4.1608,  8.1941, -2.3441, -4.4723]])
On state=1, selected action=1 , 
new state=0, done=False
action_probs_orig 
tensor([[-4.1373,  8.1354, -2.3332, -4.4445]])
On state=0, selected action=1 , 
new state=1, done=False
action_probs_orig 
tensor([[-4.